In [ ]:
import unittest
import pandas as pd
import mechanize
import zipfile
from Bio.Data.IUPACData import protein_letters_3to1
import mutfunc_functionality

In [ ]:
df = pd.read_excel("test/test_mutation_file.xlsx", header=4, keep_default_na=False)
df.head()

In [ ]:
result = mutfunc_functionality.runmutfunc("test/test_mutation_file.xlsx")

In [ ]:
result.to_csv("test/test_result_mutfunc.csv")

In [ ]:
final_df.to_csv("test/test_final_df.csv")

###Mechismo

In [ ]:
from Mechismo_functionality import run_mechismo

x = run_mechismo("test/test_mutation_file.xlsx")
x.to_csv()

In [ ]:
x.to_csv()

In [ ]:
x

CELLO2GO

In [1]:
import CellularLocation

genes = CellularLocation.locations("test/test_mutation_file.xlsx")
batch_list = CellularLocation.get_batch_list(genes, 4)

3


In [1]:
import CellularLocation
df = CellularLocation.run_cello2go("test/test_mutation_file.xlsx")

In [2]:
df

,Start POS,Cello2go probabilities,Location
0,506.0,"Extracellular,0.005,Outermembrane,0.002,Peripl...",Cytoplasmic
1,1333.0,"Extracellular,0.005,Outermembrane,0.002,Peripl...",Cytoplasmic
2,557.0,"Extracellular,0.001,Outermembrane,0.001,Peripl...",Innermembrane
3,672.0,"Extracellular,0.001,Outermembrane,0.001,Peripl...",Innermembrane
4,337.0,"Extracellular,0.006,Outermembrane,0.022,Peripl...",Innermembrane
...,...,...,...
95,333.0,"Extracellular,0.006,Outermembrane,0.022,Peripl...",Innermembrane
96,236.0,"Extracellular,0.006,Outermembrane,0.022,Peripl...",Innermembrane
97,634.0,"Extracellular,0.001,Outermembrane,0.001,Peripl...",Innermembrane
98,806.0,"Extracellular,0.001,Outermembrane,0.001,Peripl...",Innermembrane


In [ ]:
text_file = open("test/cello_results.txt", "w")
text_file.write(list)
text_file.close()

Add Experiment Test

In [ ]:
import Addexperiment
Addexperiment.add_mutation_to_experiment("test/test_mutation_file.xlsx")